In [ ]:
import requests

api_key = '8b4757e62f99439d93678edeebb440e7'
url = 'https://api.rentcast.io/v1/properties'


cities = [
    {"city": "Austin", "state": "TX"},
    {"city": "Seattle", "state": "WA"},
    {"city": "Phoenix", "state": "AZ"},
    {"city": "Miami", "state": "FL"}
]

headers = {
    'Accept': 'application/json',
    'X-API-Key': api_key
}

response = requests.get(url, headers=headers, params=params)


if response.status_code == 200:
    data = response.json()
    print("Success! Here's a sample of the data:")
    print(data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success! Here's a sample of the data:
[{'id': '1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043', 'formattedAddress': '1600 Amphitheatre Pkwy, Mountain View, CA 94043', 'addressLine1': '1600 Amphitheatre Pkwy', 'addressLine2': None, 'city': 'Mountain View', 'state': 'CA', 'zipCode': '94043', 'county': 'Santa Clara', 'latitude': 37.423099, 'longitude': -122.082395, 'propertyType': 'Manufactured', 'assessorID': '116-14-638', 'legalDescription': 'CITY/MUNI/TWP:MOUNTAIN VIEW', 'features': {'architectureType': 'Mobile Home'}, 'taxAssessments': {'2023': {'year': 2023, 'value': 96575, 'improvements': 96575}, '2024': {'year': 2024, 'value': 98506, 'improvements': 98506}}, 'propertyTaxes': {'2023': {'year': 2023, 'total': 1069}, '2024': {'year': 2024, 'total': 1085}}, 'owner': {'names': ['Elizabeth Medina'], 'type': 'Individual', 'mailingAddress': {'id': '1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043', 'formattedAddress': '1600 Amphitheatre Pkwy, Mountain View, CA 94043', 'addressLine1': '1600

In [10]:
print(type(data))

<class 'list'>


In [11]:
print(data[0].keys()) # Shows all the top-level field names inside that dictionary.
print(data[0]) # This accesses the first dictionary in the list.

dict_keys(['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'city', 'state', 'zipCode', 'county', 'latitude', 'longitude', 'propertyType', 'assessorID', 'legalDescription', 'features', 'taxAssessments', 'propertyTaxes', 'owner', 'ownerOccupied'])
{'id': '1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043', 'formattedAddress': '1600 Amphitheatre Pkwy, Mountain View, CA 94043', 'addressLine1': '1600 Amphitheatre Pkwy', 'addressLine2': None, 'city': 'Mountain View', 'state': 'CA', 'zipCode': '94043', 'county': 'Santa Clara', 'latitude': 37.423099, 'longitude': -122.082395, 'propertyType': 'Manufactured', 'assessorID': '116-14-638', 'legalDescription': 'CITY/MUNI/TWP:MOUNTAIN VIEW', 'features': {'architectureType': 'Mobile Home'}, 'taxAssessments': {'2023': {'year': 2023, 'value': 96575, 'improvements': 96575}, '2024': {'year': 2024, 'value': 98506, 'improvements': 98506}}, 'propertyTaxes': {'2023': {'year': 2023, 'total': 1069}, '2024': {'year': 2024, 'total': 1085}}, 'owner': {'nam

In [12]:
import pandas as pd

# Calling the first dictionary.
property = data[0]

# Building a dictionary of selected fields.

record = {
    'property_id': property['id'],
    'formatted_address': property['formattedAddress'],
    'address_line': property['addressLine1'],
    'city': property['city'],
    'state': property['state'],
    'zip_code': property['zipCode'],
    'county': property['county'],
    'property_type': property['propertyType'],
    'year_built': property['taxAssessments'].get('2024', {}).get('year'),
    'assessed_value': property['taxAssessments'].get('2024', {}).get('value'),
    'tax_total': property['propertyTaxes'].get('2024', {}).get('total'),
    'improvements_value': property['taxAssessments'].get('2024', {}).get('improvementsValue'),
    'year': 2024,
    'owner_name': property['owner']['names'][0] if property['owner']['names'] else None,
    'mailing_address': property['owner'].get('mailingAddress'),
    'email': property['owner'].get('email'),
    'owner_occupied': property['ownerOccupied']
}


# Converting to DataFrame
property_dim_df = pd.DataFrame([record])
property_dim_df.head()


,property_id,formatted_address,address_line,city,state,zip_code,county,property_type,year_built,assessed_value,tax_total,improvements_value,year,owner_name,mailing_address,email,owner_occupied
0,"1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043","1600 Amphitheatre Pkwy, Mountain View, CA 94043",1600 Amphitheatre Pkwy,Mountain View,CA,94043,Santa Clara,Manufactured,2024,98506,1085,None,2024,Elizabeth Medina,"{'id': '1600-Amphitheatre-Pkwy,-Mountain-View,...",None,True


In [13]:
# Ensuring decimals for fields related to money.

property_dim_df['assessed_value'] = pd.to_numeric(property_dim_df['assessed_value'], errors='coerce')
property_dim_df['tax_total'] = pd.to_numeric(property_dim_df['tax_total'], errors='coerce')

property_dim_df.head()

,property_id,formatted_address,address_line,city,state,zip_code,county,property_type,year_built,assessed_value,tax_total,improvements_value,year,owner_name,mailing_address,email,owner_occupied
0,"1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043","1600 Amphitheatre Pkwy, Mountain View, CA 94043",1600 Amphitheatre Pkwy,Mountain View,CA,94043,Santa Clara,Manufactured,2024,98506,1085,None,2024,Elizabeth Medina,"{'id': '1600-Amphitheatre-Pkwy,-Mountain-View,...",None,True


In [14]:
# Preventing duplicate rows

before = len(property_dim_df)
property_dim_df.drop_duplicates(subset='property_id', keep='last', inplace=True)
after = len(property_dim_df)
print(f"Dropped {before - after} duplicate property rows.")

property_dim_df.head()

# I'm using 'len()' to know if any duplicate row was dropped. Basically, tracking how many duplicate rows were removed.

Dropped 0 duplicate property rows.


,property_id,formatted_address,address_line,city,state,zip_code,county,property_type,year_built,assessed_value,tax_total,improvements_value,year,owner_name,mailing_address,email,owner_occupied
0,"1600-Amphitheatre-Pkwy,-Mountain-View,-CA-94043","1600 Amphitheatre Pkwy, Mountain View, CA 94043",1600 Amphitheatre Pkwy,Mountain View,CA,94043,Santa Clara,Manufactured,2024,98506,1085,None,2024,Elizabeth Medina,"{'id': '1600-Amphitheatre-Pkwy,-Mountain-View,...",None,True


In [21]:
!pip install psycopg2

In [23]:
import psycopg2

In [24]:
def get_db_connection():
    connection = psycopg2.connect(
        host="localhost",
        database="Zipco_Real_Estate_Agency",
        user="postgres",
        password="MongoDB4luv"
    )
    return connection


In [25]:
conn = get_db_connection()

In [ ]:
def load_property_records(record_list):
    conn = get_db_connection()
    cursor = conn.cursor()

    for record in record_list:
        owner_data = (
            record['owner_name'],
            record['mailing_address'],
            record['email']
        )

        cursor.execute('''
            INSERT INTO owner (name, mailing_address, email)
            VALUES (%s, %s, %s)
            RETURNING owner_id
        ''', owner_data)

        owner_id = cursor.fetchone()[0]
        

    conn.commit()
    cursor.close()
    conn.close()


In [31]:
def load_property_records(record_list):
    conn = get_db_connection()
    cursor = conn.cursor()

    for record in record_list:

        property_data = (
            record['formatted_address'],
            record['property_type'],
            record['zip_code'],
            record['county'],
            record['year_built'],
            record['assessed_value'],
            record['tax_total'],
            owner_id,
            record['owner_occupied']
        )

        cursor.execute('''
                            INSERT INTO property (formatted_address, property_type, zip_code, county, year_built, assessed_value, tax_total, owner_id, 
                            owner_occupied)
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                            RETURNING property_id
                    ''', 
                            property_data)

        property_id = cursor.fetchone()[0]

    conn.commit()
    cursor.close()
    conn.close()


In [33]:
def load_property_records(record_list):
    conn = get_db_connection()
    cursor = conn.cursor()

    for record in record_list:

        address_data = (
            record['address_line'],
            record['city'],
            record['state'],
            record['zip_code'],
            property_id
        )

        cursor.execute('''
                            INSERT INTO address (address_line, city, state, zip_code, property_id)
                            VALUES (%s, %s, %s, %s, %s)
                    ''', 
                            address_data)
        
        conn.commit()
        cursor.close()
        conn.close() 


In [34]:
def load_property_records(record_list):
    conn = get_db_connection()
    cursor = conn.cursor()

    for record in record_list:

        tax_data = (
            property_id,
            record['year'],
            record['assessed_value'],
            record['improvements_value'],
            record['tax_total']
        )

        cursor.execute('''
                            INSERT INTO tax_fact (property_id, year, assessed_value, improvements_value, total_tax)
                            VALUES (%s, %s, %s, %s, %s)
                    ''', 
                            tax_data)

        conn.commit()
        cursor.close()
        conn.close() 
